# Imports

In [78]:
import ast

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Read Files

In [104]:
df=pd.read_csv('dataset/Storico_Clienti.csv',error_bad_lines=False)#on_bad_lines='skip'
df=df.dropna(how='any')
df

b'Skipping line 225963: expected 6 fields, saw 8\n'
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3194: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ID_CLIENTE,ID_OFFERTA,mean payment delay (day),start date contract,end date contract,delayed payment
0,"""00000000""",SoloGIGA,0.9,20180213,20180413,SI
1,"""00000000""",Internet_Business,4.2,20190627,20190831,SI
2,"""00000000""",Internet_Fibra,1.9,20190324,20190526,NO
3,"""00000001""",Internet_Fibra,2.1,20171223,20180219,SI
4,"""00000001""",Internet_ADSL,1.0,20170115,20170305,SI
...,...,...,...,...,...,...
484117,"""00249996""",Internet_tutto_incluso,2.1,20170228,20170516,SI
484118,"""00249997""",SoloGIGA,0.9,20190407,20190701,NO
484119,"""00249998""",Internet_ADSL,0.0,20181024,20190120,NO
484120,"""00249999""",Internet_tutto_incluso,0.0,20190827,20191128,NO


In [10]:
dg=pd.read_csv('dataset/Anagrafica_Clienti.csv',error_bad_lines=False)#on_bad_lines='skip'
dg

b'Skipping line 117656: expected 10 fields, saw 12\n'


,ID_CLIENTE,CANALE_ATTIVAZIONE,TERRITORIO,TARGET,FLG_DOM_BANCARIA,ETA,SESSO,PAESE,DATI_PERSONALI,DATA_ATTIVAZIONE
0,"""00000000""",Web,Nord,1.0,NO,43,F,NaN,SI,20210106
1,"""00000001""",Pull,Sud_e_Isole,1.0,SI,41,M,NaN,SI,20210223
2,"""00000002""",Web,Sud_e_Isole,1.0,NO,52,M,IT,SI,20210113
3,"""00000003""",Push,Centro,1.0,NO,18,M,IT,NO,20210110
4,"""00000004""",Pull,Centro,1.0,NO,24,M,IT,SI,20210204
...,...,...,...,...,...,...,...,...,...,...
269886,"""00249995""",Pull,Sud_e_Isole,0.0,NO,45,M,IT,NaN,20210125
269887,"""00249996""",Push,Centro,0.0,SI,52,F,NaN,NO,20210102
269888,"""00249997""",Web,Centro,0.0,SI,18,M,IT,SI,20210115
269889,"""00249998""",Push,Nord,0.0,SI,76,M,IT,SI,20210204


We need to transform/merge the dataframes. since they have different number of records, we need to aggregate the first table somehow

# Feature Engineering

In [105]:
df

,ID_CLIENTE,ID_OFFERTA,mean payment delay (day),start date contract,end date contract,delayed payment
0,"""00000000""",SoloGIGA,0.9,20180213,20180413,SI
1,"""00000000""",Internet_Business,4.2,20190627,20190831,SI
2,"""00000000""",Internet_Fibra,1.9,20190324,20190526,NO
3,"""00000001""",Internet_Fibra,2.1,20171223,20180219,SI
4,"""00000001""",Internet_ADSL,1.0,20170115,20170305,SI
...,...,...,...,...,...,...
484117,"""00249996""",Internet_tutto_incluso,2.1,20170228,20170516,SI
484118,"""00249997""",SoloGIGA,0.9,20190407,20190701,NO
484119,"""00249998""",Internet_ADSL,0.0,20181024,20190120,NO
484120,"""00249999""",Internet_tutto_incluso,0.0,20190827,20191128,NO


### Payment delay statistics:

In [106]:
payment_delay_stats=df.groupby('ID_CLIENTE').agg({'mean payment delay (day)': ['mean', 'max']})
payment_delay_stats

mean payment delay (day)     
                               mean  max
ID_CLIENTE                              
"00000000"                 2.333333  4.2
"00000001"                 1.550000  2.1
"00000002"                 4.220000  8.0
"00000003"                 2.000000  2.0
"00000004"                 1.000000  1.0
...                             ...  ...
"00249995"                 0.000000  0.0
"00249996"                 2.100000  2.1
"00249997"                 0.900000  0.9
"00249998"                 0.000000  0.0
"00249999"                 0.000000  0.0

[250000 rows x 2 columns]

In [107]:
d_mergable=pd.DataFrame()
d_mergable['ID_CLIENTE']=payment_delay_stats.index.values
d_mergable['mean_pd']=payment_delay_stats[payment_delay_stats.columns[0]].values
d_mergable['max_pd']=payment_delay_stats[payment_delay_stats.columns[1]].values
d_mergable

,ID_CLIENTE,mean_pd,max_pd
0,"""00000000""",2.333333,4.2
1,"""00000001""",1.550000,2.1
2,"""00000002""",4.220000,8.0
3,"""00000003""",2.000000,2.0
4,"""00000004""",1.000000,1.0
...,...,...,...
249995,"""00249995""",0.000000,0.0
249996,"""00249996""",2.100000,2.1
249997,"""00249997""",0.900000,0.9
249998,"""00249998""",0.000000,0.0


### If had delays

In [108]:
mapping_dictionary={'SI':1,'NO':0}

In [109]:
df['delayed payment binary']=df['delayed payment'].map(mapping_dictionary)
# df.groupby(['ID_CLIENTE']).max()['delayed payment binary']

In [110]:
d_mergable['had_delays']=df.groupby('ID_CLIENTE').agg({'delayed payment binary':'max'}).values
d_mergable

,ID_CLIENTE,mean_pd,max_pd,had_delays
0,"""00000000""",2.333333,4.2,1
1,"""00000001""",1.550000,2.1,1
2,"""00000002""",4.220000,8.0,1
3,"""00000003""",2.000000,2.0,1
4,"""00000004""",1.000000,1.0,0
...,...,...,...,...
249995,"""00249995""",0.000000,0.0,1
249996,"""00249996""",2.100000,2.1,1
249997,"""00249997""",0.900000,0.9,0
249998,"""00249998""",0.000000,0.0,0


### Number of products used

In [111]:
d_mergable['num_products']=df.groupby('ID_CLIENTE').count()['ID_OFFERTA'].values
d_mergable

,ID_CLIENTE,mean_pd,max_pd,had_delays,num_products
0,"""00000000""",2.333333,4.2,1,3
1,"""00000001""",1.550000,2.1,1,2
2,"""00000002""",4.220000,8.0,1,5
3,"""00000003""",2.000000,2.0,1,1
4,"""00000004""",1.000000,1.0,0,1
...,...,...,...,...,...
249995,"""00249995""",0.000000,0.0,1,3
249996,"""00249996""",2.100000,2.1,1,1
249997,"""00249997""",0.900000,0.9,0,1
249998,"""00249998""",0.000000,0.0,0,1


In [112]:
d_mergable.to_csv('mergable.csv')

In [120]:
dg=dg.drop_duplicates()
dg=dg.merge(d_mergable,on='ID_CLIENTE')
dg

,ID_CLIENTE,CANALE_ATTIVAZIONE,TERRITORIO,TARGET,FLG_DOM_BANCARIA,ETA,SESSO,PAESE,DATI_PERSONALI,DATA_ATTIVAZIONE,mean_pd,max_pd,had_delays,num_products
0,"""00000000""",Web,Nord,1.0,NO,43,F,NaN,SI,20210106,2.333333,4.2,1,3
1,"""00000001""",Pull,Sud_e_Isole,1.0,SI,41,M,NaN,SI,20210223,1.550000,2.1,1,2
2,"""00000002""",Web,Sud_e_Isole,1.0,NO,52,M,IT,SI,20210113,4.220000,8.0,1,5
3,"""00000003""",Push,Centro,1.0,NO,18,M,IT,NO,20210110,2.000000,2.0,1,1
4,"""00000004""",Pull,Centro,1.0,NO,24,M,IT,SI,20210204,1.000000,1.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,"""00249995""",Pull,Sud_e_Isole,0.0,NO,45,M,IT,NaN,20210125,0.000000,0.0,1,3
249996,"""00249996""",Push,Centro,0.0,SI,52,F,NaN,NO,20210102,2.100000,2.1,1,1
249997,"""00249997""",Web,Centro,0.0,SI,18,M,IT,SI,20210115,0.900000,0.9,0,1
249998,"""00249998""",Push,Nord,0.0,SI,76,M,IT,SI,20210204,0.000000,0.0,0,1


In [121]:
dg.to_csv('merged_df.csv')

In [122]:
dg.corr()

,TARGET,ETA,DATA_ATTIVAZIONE,mean_pd,max_pd,had_delays,num_products
TARGET,1.000000,0.002937,0.001105,0.817814,0.803289,0.371164,0.060060
ETA,0.002937,1.000000,0.002568,0.003057,0.002819,-0.001942,-0.000417
DATA_ATTIVAZIONE,0.001105,0.002568,1.000000,-0.000524,0.000300,-0.002568,0.003312
mean_pd,0.817814,0.003057,-0.000524,1.000000,0.960003,0.343211,0.054341
max_pd,0.803289,0.002819,0.000300,0.960003,1.000000,0.380536,0.192283
had_delays,0.371164,-0.001942,-0.002568,0.343211,0.380536,1.000000,0.264504
num_products,0.060060,-0.000417,0.003312,0.054341,0.192283,0.264504,1.000000
